Amazon EG Apple

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Headers for requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

driver = webdriver.Firefox()

urls = []
for page in range(1, 3):
    url = f'https://www.amazon.eg/s?i=electronics&bbn=21832883031&rh=n%3A21832883031%2Cp_89%3AApple&page={page}&language=en_AE'

    driver.get(url)
    page_source = driver.page_source
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')))
    time.sleep(2)
    soup = BeautifulSoup(page_source, 'html.parser')
    url_tags = soup.find_all('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    for url_tag in url_tags:
        urls.append(f"https://www.amazon.eg{url_tag['href']}&language=en_AE")
driver.quit()
urls = list(set(urls))
print(f'{len(urls)} Product URLs retrieved.')

28 Product URLs retrieved.


In [7]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def create_optimized_firefox_options():
    options = Options()
    options.headless = True  # Run browser in headless mode

    # Create a Firefox profile for advanced performance optimizations
    profile = webdriver.FirefoxProfile()

    # Disable images to save bandwidth
    profile.set_preference("permissions.default.image", 2)

    # Disable CSS to speed up page loading (if CSS is not needed)
    profile.set_preference("permissions.default.stylesheet", 2)

    # Disable JavaScript for speed (if JavaScript is not needed for scraping)
    profile.set_preference("javascript.enabled", False)

    # Disable plugins and Flash to reduce resource usage
    profile.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", False)
    profile.set_preference("media.autoplay.default", 0)  # Disable media autoplay

    # Disable WebRTC to minimize network-related overhead
    profile.set_preference("media.peerconnection.enabled", False)

    # Block tracking content and ads
    profile.set_preference("privacy.trackingprotection.enabled", True)
    profile.set_preference("network.http.referer.spoofSource", True)
    profile.set_preference("privacy.trackingprotection.socialtracking.enabled", True)

    # Disable media and video buffering
    profile.set_preference("media.block-autoplay-until-in-foreground", True)
    profile.set_preference("media.autoplay.default", 5)
    profile.set_preference("permissions.default.media", 2)

    # Turn off telemetry to avoid sending data back to Mozilla
    profile.set_preference("datareporting.healthreport.uploadEnabled", False)
    profile.set_preference("toolkit.telemetry.reportingpolicy.firstRun", False)

    # Suppress unnecessary logging to speed up execution
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    options.log.level = "error"  # Minimize logs

    # Attach the profile to Firefox options
    options.profile = profile

    return options  # To run Firefox in headless mode

options = create_optimized_firefox_options()
titles = []
prices = []
rates = []
brands = []
OSs = []
rams = []
cpus = []
storages = []
SSs = []
resolutions = []
refresh_rate= []
model_names = []
wireless_providers = []
colors = []
imgs = []
reviews = []
cpu_speeds = []
Cellular_technology = []
SIMs = []
WTs =[]
camera =[]
beforeprice=[]
Currency = []
Website =[]
def scrape_amazon_reviews(url):
    # Fetch the page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return []
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the reviews section
    reviews_section = soup.find('div', id='cm-cr-dp-review-list')
    if not reviews_section:
        print("Reviews section not found")
        return []
    
    reviews = reviews_section.find_all('div', {'data-hook': 'review'})

    review_data = []
    for review in reviews:
        # Find reviewer name
        reviewer_name = review.find('span', class_='a-profile-name')
        reviewer_name = reviewer_name.text.strip() if reviewer_name else 'N/A'
        
        # Find review rating
        review_rating = review.find('i', {'data-hook': 'review-star-rating'})
        review_rating = review_rating.text.strip() if review_rating else 'N/A'
        
        # Find review title
        review_title = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title.text.strip() if review_title else 'N/A'
        
        # Find review text
        review_text = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text.text.strip() if review_text else 'N/A'
        
        # Find review date
        review_date = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date.text.strip() if review_date else 'N/A'

        review_data.append({
            'reviewer_name': reviewer_name,
            'review_rating': review_rating,
            'review_title': review_title,
            'review_text': review_text,
            'review_date': review_date
        })

    return review_data

def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None

def scrape_product_data(url):
    try:
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-size-large.product-title-word-break')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        camera_pattern = re.compile(r'(\d+MP)\s+MAIN CAMERA')
        camera_pattern2 = re.compile(r'(\d+MP)\s+camera')
        camera_pattern3 = re.compile(r'(\d+ MP)\s+camera')
        camera_pattern4 = re.compile(r'(\d+ MP)\s+MAIN CAMERA')
        camera_info = None
        for li in soup.find_all('li', class_='a-spacing-mini'):
            match = camera_pattern.search(li.text)
            match2 = camera_pattern2.search(li.text)
            match3 = camera_pattern3.search(li.text)
            match4 = camera_pattern4.search(li.text)
            if match:
                camera_info = match.group(1)
                break
            if match2:
                camera_info = match2.group(1)
                break
            if match3:
                camera_info = match3.group(1)
                break
            if match4:
                camera_info = match4.group(1)
                break
        camera_features = None  # Placeholder for camera information
        
        # Find the table with camera features
        camera_features_row = soup.find('table', id='productDetails_techSpec_section_1')
        if camera_features_row:
            # Find the specific row containing "Other camera features"
            camera_features_row2 = camera_features_row.find('th', string='Other camera features')
            
            # Ensure the row exists and extract the camera details
            if camera_features_row2:
                camera_features = camera_features_row2.find_next('td').get_text(strip=True)
                
                # Extract the main camera's MP (assuming 'XXMP main camera' format)
                if 'main camera' in camera_features:
                    # Get the part before "main camera" and extract the megapixels
                    camera_info = camera_features.split('main camera')[0].strip().split()[-1]
        
        color_row = soup.find('tr', class_='a-spacing-small po-color')
        color = None  # Initialize color
        
        if color_row:
            # If the color row exists, find the span inside it and get the text
            color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
        
        elif soup.find('table', id ='productDetails_techSpec_section_1'):
            color_row2 = soup.find('th', string='Color')
            if color_row2:
                color = color_row2.find_next('td').get_text(strip=True)
        
        else:
            # Check for color in div with id='variation_color_name'
            outerSelect = soup.find('div', id='variation_color_name')
            if outerSelect:
                selection_row = outerSelect.find('span', class_='selection')
                if selection_row:
                    color = get_text_or_none(selection_row)
        price = get_text_or_none(soup.find('div', class_='a-section a-spacing-micro').find('span', class_='a-offscreen')
                         if soup.find('div', class_='a-section a-spacing-micro') else None)
        if not price:
            price = get_text_or_none(soup.find('span', class_='a-price-whole'))



        # Scrape data using helper functon
        data = {
            'Title': get_text_or_none(soup.find('span', class_='a-size-large product-title-word-break')),
            'Brand': get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
            'OS': get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
            'RAM': get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
            'CPU': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
            'Storage': get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
            'Screen Size': get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
            'Resolution': get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
            'CPU Speed': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
            'Model': get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
            'Wireless Provider': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
            'Cellular Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
            'Color': color,
            'Refresh Rate': get_text_or_none(soup.find('tr', class_='a-spacing-small po-refresh_rate').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-refresh_rate') else None),
            'SIM Count' : get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
            'Wireless Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
            'Price': price,
            'Price Before Promotion' : get_text_or_none(soup.find('span', class_='a-price a-text-price').find('span', class_='a-offscreen') if soup.find('span', class_='a-price a-text-price') else None),
            'Rate': get_text_or_none(soup.find('span', class_='a-icon-alt')),
            'Camera': camera_info,
            'IMG': get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
            'Reviews': scrape_amazon_reviews(url)
        }
        driver.quit()
        return data

    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")
        return None

# Scrape product data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_product_data, urls)
    for result in results:
        if result:
            titles.append(result['Title'])
            brands.append(result['Brand'])
            OSs.append(result['OS'])
            rams.append(result['RAM'])
            cpus.append(result['CPU'])
            storages.append(result['Storage'])
            SSs.append(result['Screen Size'])
            resolutions.append(result['Resolution'])
            refresh_rate.append(result['Refresh Rate'])
            cpu_speeds.append(result['CPU Speed'])
            model_names.append(result['Model'])
            wireless_providers.append(result['Wireless Provider'])
            Cellular_technology.append(result['Cellular Technology'])
            WTs.append(result['Wireless Technology'])
            SIMs.append(result['SIM Count'])
            colors.append(result['Color'])
            prices.append(result['Price'])
            beforeprice.append(result['Price Before Promotion'])
            camera.append(result['Camera'])
            rates.append(result['Rate'])
            imgs.append(result['IMG'])
            reviews.append(result['Reviews'])
            Currency.append('EGP')
            Website.append('Amazon EG')

# Create a DataFrame to store the scraped data
df_apple = pd.DataFrame({
    'Title': titles,
    'Brand': brands,
    'OS': OSs,
    'RAM': rams,
    'CPU': cpus,
    'Storage': storages,
    'Screen Size': SSs,
    'Resolution': resolutions,
    'CPU Speed': cpu_speeds,
    'Refresh Rate':refresh_rate,
    'Model': model_names,
    'Wireless Technology' : WTs,
    'SIM Count' : SIMs , 
    'Wireless Provider': wireless_providers,
    'Cellular Technology': Cellular_technology,
    'Camera' : camera,
    'Color': colors,
    'Price': prices,
    'Price Before Promotion':beforeprice,
    'Currancy' : Currency,
    'Rate': rates,
    'URL' : urls,
    'IMG': imgs,
    'Reviews': reviews,
    'Website' : Website
})

df_apple

Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found


,Title,Brand,OS,RAM,CPU,Storage,Screen Size,Resolution,CPU Speed,Refresh Rate,...,Camera,Color,Price,Price Before Promotion,Currancy,Rate,URL,IMG,Reviews,Website
0,"Apple iPhone 13 with FaceTime - 128GB, 4GB RAM...",Apple,ios,4 GB,A13,128 GB,6.1 Inches,3840 x 2160,None,60 Hz,...,None,None,"3,464.",None,EGP,3.9 out of 5 stars,https://www.amazon.eg/-/en/Apple-iPhone-FaceTi...,https://m.media-amazon.com/images/I/71GLMJ7TQi...,[],Amazon EG
1,"New Apple iPhone 14 Plus (128 GB) - Blue, Blue...",Apple,iOS 16,None,A13,128 GB,6.7 Inches,2160 x 1080,None,120 Hz,...,None,None,"EGP35,450.00",None,EGP,4.5 out of 5 stars,https://www.amazon.eg/-/en/New-Apple-iPhone-Pl...,https://m.media-amazon.com/images/I/611d1GHKz-...,"[{'reviewer_name': 'Ebraam', 'review_rating': ...",Amazon EG
2,Apple iPhone 13 (128 GB) - Green,Apple,iOS 15,None,None,128 GB,6.1 Inches,None,None,60 Hz,...,None,Green,"EGP29,333.00","EGP30,750.00",EGP,4.4 out of 5 stars,https://www.amazon.eg/-/en/Apple-iPhone-13-128...,https://m.media-amazon.com/images/I/51yG1FjZVY...,"[{'reviewer_name': 'khaled', 'review_rating': ...",Amazon EG
3,Iphone 13 International Green,Apple,Iphone 13,4 GB,A13,128 GB,6.1 Inches,None,2 GHz,60 Hz,...,None,None,"EGP30,500.00",None,EGP,4.4 out of 5 stars,https://www.amazon.eg/-/en/Apple-MNGG3J-A-Ipho...,https://m.media-amazon.com/images/I/61qr5N7ynX...,"[{'reviewer_name': 'Mohanad Helmy', 'review_ra...",Amazon EG
4,"Apple iPhone 14 (128 GB) - Midnight, Bluetooth...",Apple,iOS 16,6 GB,A13,128 GB,6.1 Inches,720 x 1600,None,60 Hz,...,None,None,"EGP34,550.00",None,EGP,4.5 out of 5 stars,https://www.amazon.eg/-/en/Apple-iPhone-14-128...,https://m.media-amazon.com/images/I/51sFkTjuc3...,"[{'reviewer_name': 'user-MW1F7K1', 'review_rat...",Amazon EG
5,"Apple iPhone 11 with FaceTime - 128GB, 4GB RAM...",Apple,iOS 12,4 GB,A13,128 GB,6.1 Inches,1280 x 720,None,60 Hz,...,12MP,None,"EGP21,550.00",None,EGP,4.4 out of 5 stars,https://www.amazon.eg/-/en/Apple-iPhone-11-Fac...,https://m.media-amazon.com/images/I/71kZfQA-Y7...,"[{'reviewer_name': 'Samir', 'review_rating': '...",Amazon EG
6,"Apple iPhone 11 with FaceTime - 128GB, 4GB RAM...",Apple,iOS 13,4 GB,A13,128 GB,6.1 Inches,828 x 1792,None,60 Hz,...,None,white,"EGP21,550.00",None,EGP,4.5 out of 5 stars,https://www.amazon.eg/-/en/Apple-iPhone-11-Fac...,https://m.media-amazon.com/images/I/71QE00iB9I...,"[{'reviewer_name': 'user-MY1CW6', 'review_rati...",Amazon EG
7,Apple iPhone 15 (128 GB) - Black,Apple,iOS 17,None,None,128 GB,6.1,None,None,None,...,48MP,Black,"EGP38,800.00",None,EGP,4.7 out of 5 stars,https://www.amazon.eg/-/en/Apple-iPhone-15-128...,https://m.media-amazon.com/images/I/61qQCRAQ3s...,[],Amazon EG
8,Apple iPhone 15 (128 GB) - Blue,Apple,iOS 15,None,A13,128 GB,6.1 Inches,None,2 GHz,120 Hz,...,None,None,"EGP38,550.00","EGP39,896.00",EGP,4.2 out of 5 stars,https://www.amazon.eg/-/en/Apple-iPhone-15-128...,https://m.media-amazon.com/images/I/71k38LlSqM...,"[{'reviewer_name': 'Wafaa', 'review_rating': '...",Amazon EG
9,"Apple iPhone 13 with FaceTime - 128GB, 4GB RAM...",Apple,ios,4 GB,A13,128 GB,6.1 Inches,1170 x 2532,None,60 Hz,...,None,None,"2,599.",None,EGP,3.6 out of 5 stars,https://www.amazon.eg/-/en/Apple-iPhone-FaceTi...,https://m.media-amazon.com/images/I/61VuVU94Rn...,[],Amazon EG


In [8]:
import pandas as pd
import re
allcolors = [
     "Rose Gold", "Navy Blue","Light Blue","Cobalt Violet","Awesome Graphite","Titanium Black","Wave Green"
    "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink","Carbon Grey",
    "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black","Cobalt Violet",
    "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
    "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White","Moonlight White"
    "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", "Awesome Navy"
    "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", "Awesome Lime","Silver Shadow",
    "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
    "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", "Clear","Navy","Chartreuse",
    "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", "Graphite","Violet",
    "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
]
import pandas as pd
def find_color_in_title(title, allcolors):
    for color in allcolors:
        if color.lower() in title.lower():
            return color
    return None
df_apple['Color'] = df_apple.apply(lambda row: find_color_in_title(row['Title'], allcolors) or row['Color'], axis=1)

In [13]:
df_apple.to_csv('Apple EG.csv', index = False)

Amazon EG Samsung

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Headers for requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

driver = webdriver.Firefox()

urls = []
for page in range(1, 4):
    url = f'https://www.amazon.eg/s?i=electronics&bbn=21832883031&rh=n%3A21832883031%2Cp_89%3ASAMSUNG&page={page}&language=en_AE'

    driver.get(url)
    page_source = driver.page_source
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')))
    time.sleep(2)
    soup = BeautifulSoup(page_source, 'html.parser')
    url_tags = soup.find_all('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    for url_tag in url_tags:
        urls.append(f"https://www.amazon.eg{url_tag['href']}&language=en_AE")
driver.quit()
urls = list(set(urls))
print(f'{len(urls)} Product URLs retrieved.')

57 Product URLs retrieved.


In [11]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def create_optimized_firefox_options():
    options = Options()
    options.headless = True  # Run browser in headless mode

    # Create a Firefox profile for advanced performance optimizations
    profile = webdriver.FirefoxProfile()

    # Disable images to save bandwidth
    profile.set_preference("permissions.default.image", 2)

    # Disable CSS to speed up page loading (if CSS is not needed)
    profile.set_preference("permissions.default.stylesheet", 2)

    # Disable JavaScript for speed (if JavaScript is not needed for scraping)
    profile.set_preference("javascript.enabled", False)

    # Disable plugins and Flash to reduce resource usage
    profile.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", False)
    profile.set_preference("media.autoplay.default", 0)  # Disable media autoplay

    # Disable WebRTC to minimize network-related overhead
    profile.set_preference("media.peerconnection.enabled", False)

    # Block tracking content and ads
    profile.set_preference("privacy.trackingprotection.enabled", True)
    profile.set_preference("network.http.referer.spoofSource", True)
    profile.set_preference("privacy.trackingprotection.socialtracking.enabled", True)

    # Disable media and video buffering
    profile.set_preference("media.block-autoplay-until-in-foreground", True)
    profile.set_preference("media.autoplay.default", 5)
    profile.set_preference("permissions.default.media", 2)

    # Turn off telemetry to avoid sending data back to Mozilla
    profile.set_preference("datareporting.healthreport.uploadEnabled", False)
    profile.set_preference("toolkit.telemetry.reportingpolicy.firstRun", False)

    # Suppress unnecessary logging to speed up execution
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    options.log.level = "error"  # Minimize logs

    # Attach the profile to Firefox options
    options.profile = profile

    return options  # To run Firefox in headless mode

options = create_optimized_firefox_options()
titles = []
prices = []
rates = []
brands = []
OSs = []
rams = []
cpus = []
storages = []
SSs = []
resolutions = []
refresh_rate= []
model_names = []
wireless_providers = []
colors = []
imgs = []
reviews = []
cpu_speeds = []
Cellular_technology = []
SIMs = []
WTs =[]
camera =[]
beforeprice=[]
Currency = []
Website =[]
def scrape_amazon_reviews(url):
    # Fetch the page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return []
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the reviews section
    reviews_section = soup.find('div', id='cm-cr-dp-review-list')
    if not reviews_section:
        print("Reviews section not found")
        return []
    
    reviews = reviews_section.find_all('div', {'data-hook': 'review'})

    review_data = []
    for review in reviews:
        # Find reviewer name
        reviewer_name = review.find('span', class_='a-profile-name')
        reviewer_name = reviewer_name.text.strip() if reviewer_name else 'N/A'
        
        # Find review rating
        review_rating = review.find('i', {'data-hook': 'review-star-rating'})
        review_rating = review_rating.text.strip() if review_rating else 'N/A'
        
        # Find review title
        review_title = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title.text.strip() if review_title else 'N/A'
        
        # Find review text
        review_text = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text.text.strip() if review_text else 'N/A'
        
        # Find review date
        review_date = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date.text.strip() if review_date else 'N/A'

        review_data.append({
            'reviewer_name': reviewer_name,
            'review_rating': review_rating,
            'review_title': review_title,
            'review_text': review_text,
            'review_date': review_date
        })

    return review_data

def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None

def scrape_product_data(url):
    try:
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-size-large.product-title-word-break')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        camera_pattern = re.compile(r'(\d+MP)\s+MAIN CAMERA')
        camera_pattern2 = re.compile(r'(\d+MP)\s+camera')
        camera_pattern3 = re.compile(r'(\d+ MP)\s+camera')
        camera_pattern4 = re.compile(r'(\d+ MP)\s+MAIN CAMERA')
        camera_info = None
        for li in soup.find_all('li', class_='a-spacing-mini'):
            match = camera_pattern.search(li.text)
            match2 = camera_pattern2.search(li.text)
            match3 = camera_pattern3.search(li.text)
            match4 = camera_pattern4.search(li.text)
            if match:
                camera_info = match.group(1)
                break
            if match2:
                camera_info = match2.group(1)
                break
            if match3:
                camera_info = match3.group(1)
                break
            if match4:
                camera_info = match4.group(1)
                break
        camera_features = None  # Placeholder for camera information
        
        # Find the table with camera features
        camera_features_row = soup.find('table', id='productDetails_techSpec_section_1')
        if camera_features_row:
            # Find the specific row containing "Other camera features"
            camera_features_row2 = camera_features_row.find('th', string='Other camera features')
            
            # Ensure the row exists and extract the camera details
            if camera_features_row2:
                camera_features = camera_features_row2.find_next('td').get_text(strip=True)
                
                # Extract the main camera's MP (assuming 'XXMP main camera' format)
                if 'main camera' in camera_features:
                    # Get the part before "main camera" and extract the megapixels
                    camera_info = camera_features.split('main camera')[0].strip().split()[-1]
        
        color_row = soup.find('tr', class_='a-spacing-small po-color')
        color = None  # Initialize color
        
        if color_row:
            # If the color row exists, find the span inside it and get the text
            color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
        
        elif soup.find('table', id ='productDetails_techSpec_section_1'):
            color_row2 = soup.find('th', string='Color')
            if color_row2:
                color = color_row2.find_next('td').get_text(strip=True)
        
        else:
            # Check for color in div with id='variation_color_name'
            outerSelect = soup.find('div', id='variation_color_name')
            if outerSelect:
                selection_row = outerSelect.find('span', class_='selection')
                if selection_row:
                    color = get_text_or_none(selection_row)
        price = get_text_or_none(soup.find('div', class_='a-section a-spacing-micro').find('span', class_='a-offscreen')
                         if soup.find('div', class_='a-section a-spacing-micro') else None)
        if not price:
            price = get_text_or_none(soup.find('span', class_='a-price-whole'))



        # Scrape data using helper functon
        data = {
            'Title': get_text_or_none(soup.find('span', class_='a-size-large product-title-word-break')),
            'Brand': get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
            'OS': get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
            'RAM': get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
            'CPU': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
            'Storage': get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
            'Screen Size': get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
            'Resolution': get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
            'CPU Speed': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
            'Model': get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
            'Wireless Provider': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
            'Cellular Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
            'Color': color,
            'Refresh Rate': get_text_or_none(soup.find('tr', class_='a-spacing-small po-refresh_rate').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-refresh_rate') else None),
            'SIM Count' : get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
            'Wireless Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
            'Price': price,
            'Price Before Promotion' : get_text_or_none(soup.find('span', class_='a-price a-text-price').find('span', class_='a-offscreen') if soup.find('span', class_='a-price a-text-price') else None),
            'Rate': get_text_or_none(soup.find('span', class_='a-icon-alt')),
            'Camera': camera_info,
            'IMG': get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
            'Reviews': scrape_amazon_reviews(url)
        }
        driver.quit()
        return data

    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")
        return None

# Scrape product data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_product_data, urls)
    for result in results:
        if result:
            titles.append(result['Title'])
            brands.append(result['Brand'])
            OSs.append(result['OS'])
            rams.append(result['RAM'])
            cpus.append(result['CPU'])
            storages.append(result['Storage'])
            SSs.append(result['Screen Size'])
            resolutions.append(result['Resolution'])
            refresh_rate.append(result['Refresh Rate'])
            cpu_speeds.append(result['CPU Speed'])
            model_names.append(result['Model'])
            wireless_providers.append(result['Wireless Provider'])
            Cellular_technology.append(result['Cellular Technology'])
            WTs.append(result['Wireless Technology'])
            SIMs.append(result['SIM Count'])
            colors.append(result['Color'])
            prices.append(result['Price'])
            beforeprice.append(result['Price Before Promotion'])
            camera.append(result['Camera'])
            rates.append(result['Rate'])
            imgs.append(result['IMG'])
            reviews.append(result['Reviews'])
            Currency.append('EGP')
            Website.append('Amazon EG')

# Create a DataFrame to store the scraped data
df_samsung = pd.DataFrame({
    'Title': titles,
    'Brand': brands,
    'OS': OSs,
    'RAM': rams,
    'CPU': cpus,
    'Storage': storages,
    'Screen Size': SSs,
    'Resolution': resolutions,
    'CPU Speed': cpu_speeds,
    'Refresh Rate':refresh_rate,
    'Model': model_names,
    'Wireless Technology' : WTs,
    'SIM Count' : SIMs , 
    'Wireless Provider': wireless_providers,
    'Cellular Technology': Cellular_technology,
    'Camera' : camera,
    'Color': colors,
    'Price': prices,
    'Price Before Promotion':beforeprice,
    'Currancy' : Currency,
    'Rate': rates,
    'URL' : urls,
    'IMG': imgs,
    'Reviews': reviews,
    'Website' : Website
})

df_samsung

Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found


,Title,Brand,OS,RAM,CPU,Storage,Screen Size,Resolution,CPU Speed,Refresh Rate,...,Camera,Color,Price,Price Before Promotion,Currancy,Rate,URL,IMG,Reviews,Website
0,"SAMSUNG Galaxy A13 LTE Android, 64GB, 4GB RAM,...",SAMSUNG,None,4 GB,A13,64 GB,6.6 Inches,1080 x 2408,None,60 Hz,...,None,clear,"EGP6,250.00",None,EGP,4.4 out of 5 stars,https://www.amazon.eg/-/en/SAMSUNG-Galaxy-Andr...,https://m.media-amazon.com/images/I/714how2Z6r...,"[{'reviewer_name': 'محمد حسن محمد', 'review_ra...",Amazon EG
1,"Samsung Galaxy A23 Dual SIM, 6GB RAM, 128GB 4G...",SAMSUNG,Android 12.0,6 GB,Snapdragon,128 GB,6.4 Inches,1080 x 2408,1.6 GHz,90 Hz,...,None,None,"7,865.",None,EGP,4.3 out of 5 stars,https://www.amazon.eg/-/en/Samsung-Galaxy-A23-...,https://m.media-amazon.com/images/I/61mOx8va+L...,"[{'reviewer_name': 'Abdelraheem', 'review_rati...",Amazon EG
2,"SAMSUNG Galaxy Z Flip6, AI Phone, 12GB RAM, 25...",SAMSUNG,None,None,None,256 GB,None,None,None,None,...,50MP,Mint,"EGP41,888.00",None,EGP,3.0 out of 5 stars,https://www.amazon.eg/-/en/SAMSUNG-Storage-And...,https://m.media-amazon.com/images/I/61-4H4B2qs...,[],Amazon EG
3,"SAMSUNG Galaxy S24 AI Bundle (Smartphone, 256G...",SAMSUNG,Android 10.0,8 GB,Snapdragon,256 GB,6.8 Inches,None,None,120 Hz,...,None,None,"EGP44,900.00",None,EGP,Previous page,https://www.amazon.eg/-/en/SAMSUNG-Galaxy-Smar...,https://m.media-amazon.com/images/I/41PK-A10fz...,[],Amazon EG
4,"SAMSUNG Galaxy S24 Ultra, AI Phone, 1TB Storag...",SAMSUNG,Android,12 GB,None,1 TB,6.8,1440 x 3120,None,120 Hz,...,None,None,"EGP88,500.00",None,EGP,5.0 out of 5 stars,https://www.amazon.eg/-/en/SAMSUNG-Storage-Tit...,https://m.media-amazon.com/images/I/71WkqFssob...,[],Amazon EG
5,"Samsung Galaxy Z Fold5 Folding Phone, Android ...",SAMSUNG,Android 10.0,12 GB,Snapdragon,512 GB,7.6 Inches,1080 x 2400,None,120 Hz,...,None,None,"EGP65,550.00",None,EGP,5.0 out of 5 stars,https://www.amazon.eg/-/en/Samsung-Folding-Sma...,https://m.media-amazon.com/images/I/71Cx1wmgQl...,[],Amazon EG
6,"Samsung Galaxy A55 5G, Android Smartphone, Dua...",SAMSUNG,Android 11.0,8 GB,Snapdragon,256 GB,6.5,1080 x 2340,2.3 GHz,None,...,None,None,"EGP17,638.00","EGP23,499.00",EGP,4.2 out of 5 stars,https://www.amazon.eg/-/en/Samsung-Android-Sma...,https://m.media-amazon.com/images/I/71iDfC-A+G...,"[{'reviewer_name': 'Ahmed S.', 'review_rating'...",Amazon EG
7,"SAMSUNG Galaxy S24+, AI Android Smartphone, 25...",SAMSUNG,Android,12 GB,Snapdragon,256 GB,6.7 Inches,1440 x 3120,1.8 GHz,492 Hz,...,None,None,"EGP42,450.00","EGP50,900.00",EGP,4.2 out of 5 stars,https://www.amazon.eg/-/en/SAMSUNG-Android-Sma...,https://m.media-amazon.com/images/I/61wfSxvntT...,"[{'reviewer_name': 'Ahmed', 'review_rating': '...",Amazon EG
8,"Samsung Galaxy A55 5G, Android Smartphone, Dua...",SAMSUNG,Android 11.0,8 GB,Snapdragon,256 GB,6.5,None,2.3 GHz,None,...,None,None,"EGP19,999.00","EGP23,499.00",EGP,3.9 out of 5 stars,https://www.amazon.eg/-/en/Samsung-Android-Sma...,https://m.media-amazon.com/images/I/71Azxw1VWe...,"[{'reviewer_name': 'mahmoud kaliouby', 'review...",Amazon EG
9,"SAMSUNG Galaxy S24 AI Bundle (Smartphone, 256G...",SAMSUNG,Android 10.0,8 GB,Snapdragon,256 GB,None,None,None,120 Hz,...,None,Cobalt Violet,"EGP37,750.00",None,EGP,Previous page,https://www.amazon.eg/-/en/SAMSUNG-Galaxy-Smar...,https://m.media-amazon.com/images/I/41R-jKCABO...,[],Amazon EG


In [12]:
import pandas as pd
import re
allcolors = [
     "Rose Gold", "Navy Blue","Light Blue","Cobalt Violet","Awesome Graphite","Titanium Black","Wave Green"
    "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink","Carbon Grey",
    "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black","Cobalt Violet",
    "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
    "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White","Moonlight White"
    "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", "Awesome Navy"
    "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", "Awesome Lime","Silver Shadow",
    "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
    "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", "Clear","Navy","Chartreuse",
    "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", "Graphite","Violet",
    "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
]
import pandas as pd
def find_color_in_title(title, allcolors):
    for color in allcolors:
        if color.lower() in title.lower():
            return color
    return None
df_samsung['Color'] = df_samsung.apply(lambda row: find_color_in_title(row['Title'], allcolors) or row['Color'], axis=1)

In [14]:
df_samsung.to_csv('Samsung EG.csv', index = False)

Amazon EG XIAOMI

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Headers for requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

driver = webdriver.Firefox()

urls = []
for page in range(1, 3):
    url = f'https://www.amazon.eg/s?i=electronics&bbn=21832883031&rh=n%3A21832883031%2Cp_89%3AXiaomi&page={page}&language=en_AE'

    driver.get(url)
    page_source = driver.page_source
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')))
    time.sleep(2)
    soup = BeautifulSoup(page_source, 'html.parser')
    url_tags = soup.find_all('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    for url_tag in url_tags:
        urls.append(f"https://www.amazon.eg{url_tag['href']}&language=en_AE")
driver.quit()
urls = list(set(urls))
print(f'{len(urls)} Product URLs retrieved.')

44 Product URLs retrieved.


In [16]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def create_optimized_firefox_options():
    options = Options()
    options.headless = True  # Run browser in headless mode

    # Create a Firefox profile for advanced performance optimizations
    profile = webdriver.FirefoxProfile()

    # Disable images to save bandwidth
    profile.set_preference("permissions.default.image", 2)

    # Disable CSS to speed up page loading (if CSS is not needed)
    profile.set_preference("permissions.default.stylesheet", 2)

    # Disable JavaScript for speed (if JavaScript is not needed for scraping)
    profile.set_preference("javascript.enabled", False)

    # Disable plugins and Flash to reduce resource usage
    profile.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", False)
    profile.set_preference("media.autoplay.default", 0)  # Disable media autoplay

    # Disable WebRTC to minimize network-related overhead
    profile.set_preference("media.peerconnection.enabled", False)

    # Block tracking content and ads
    profile.set_preference("privacy.trackingprotection.enabled", True)
    profile.set_preference("network.http.referer.spoofSource", True)
    profile.set_preference("privacy.trackingprotection.socialtracking.enabled", True)

    # Disable media and video buffering
    profile.set_preference("media.block-autoplay-until-in-foreground", True)
    profile.set_preference("media.autoplay.default", 5)
    profile.set_preference("permissions.default.media", 2)

    # Turn off telemetry to avoid sending data back to Mozilla
    profile.set_preference("datareporting.healthreport.uploadEnabled", False)
    profile.set_preference("toolkit.telemetry.reportingpolicy.firstRun", False)

    # Suppress unnecessary logging to speed up execution
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    options.log.level = "error"  # Minimize logs

    # Attach the profile to Firefox options
    options.profile = profile

    return options  # To run Firefox in headless mode

options = create_optimized_firefox_options()
titles = []
prices = []
rates = []
brands = []
OSs = []
rams = []
cpus = []
storages = []
SSs = []
resolutions = []
refresh_rate= []
model_names = []
wireless_providers = []
colors = []
imgs = []
reviews = []
cpu_speeds = []
Cellular_technology = []
SIMs = []
WTs =[]
camera =[]
beforeprice=[]
Currency = []
Website =[]
def scrape_amazon_reviews(url):
    # Fetch the page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return []
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the reviews section
    reviews_section = soup.find('div', id='cm-cr-dp-review-list')
    if not reviews_section:
        print("Reviews section not found")
        return []
    
    reviews = reviews_section.find_all('div', {'data-hook': 'review'})

    review_data = []
    for review in reviews:
        # Find reviewer name
        reviewer_name = review.find('span', class_='a-profile-name')
        reviewer_name = reviewer_name.text.strip() if reviewer_name else 'N/A'
        
        # Find review rating
        review_rating = review.find('i', {'data-hook': 'review-star-rating'})
        review_rating = review_rating.text.strip() if review_rating else 'N/A'
        
        # Find review title
        review_title = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title.text.strip() if review_title else 'N/A'
        
        # Find review text
        review_text = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text.text.strip() if review_text else 'N/A'
        
        # Find review date
        review_date = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date.text.strip() if review_date else 'N/A'

        review_data.append({
            'reviewer_name': reviewer_name,
            'review_rating': review_rating,
            'review_title': review_title,
            'review_text': review_text,
            'review_date': review_date
        })

    return review_data

def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None

def scrape_product_data(url):
    try:
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-size-large.product-title-word-break')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        camera_pattern = re.compile(r'(\d+MP)\s+MAIN CAMERA')
        camera_pattern2 = re.compile(r'(\d+MP)\s+camera')
        camera_pattern3 = re.compile(r'(\d+ MP)\s+camera')
        camera_pattern4 = re.compile(r'(\d+ MP)\s+MAIN CAMERA')
        camera_info = None
        for li in soup.find_all('li', class_='a-spacing-mini'):
            match = camera_pattern.search(li.text)
            match2 = camera_pattern2.search(li.text)
            match3 = camera_pattern3.search(li.text)
            match4 = camera_pattern4.search(li.text)
            if match:
                camera_info = match.group(1)
                break
            if match2:
                camera_info = match2.group(1)
                break
            if match3:
                camera_info = match3.group(1)
                break
            if match4:
                camera_info = match4.group(1)
                break
        camera_features = None  # Placeholder for camera information
        
        # Find the table with camera features
        camera_features_row = soup.find('table', id='productDetails_techSpec_section_1')
        if camera_features_row:
            # Find the specific row containing "Other camera features"
            camera_features_row2 = camera_features_row.find('th', string='Other camera features')
            
            # Ensure the row exists and extract the camera details
            if camera_features_row2:
                camera_features = camera_features_row2.find_next('td').get_text(strip=True)
                
                # Extract the main camera's MP (assuming 'XXMP main camera' format)
                if 'main camera' in camera_features:
                    # Get the part before "main camera" and extract the megapixels
                    camera_info = camera_features.split('main camera')[0].strip().split()[-1]
        
        color_row = soup.find('tr', class_='a-spacing-small po-color')
        color = None  # Initialize color
        
        if color_row:
            # If the color row exists, find the span inside it and get the text
            color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
        
        elif soup.find('table', id ='productDetails_techSpec_section_1'):
            color_row2 = soup.find('th', string='Color')
            if color_row2:
                color = color_row2.find_next('td').get_text(strip=True)
        
        else:
            # Check for color in div with id='variation_color_name'
            outerSelect = soup.find('div', id='variation_color_name')
            if outerSelect:
                selection_row = outerSelect.find('span', class_='selection')
                if selection_row:
                    color = get_text_or_none(selection_row)
        price = get_text_or_none(soup.find('div', class_='a-section a-spacing-micro').find('span', class_='a-offscreen')
                         if soup.find('div', class_='a-section a-spacing-micro') else None)
        if not price:
            price = get_text_or_none(soup.find('span', class_='a-price-whole'))



        # Scrape data using helper functon
        data = {
            'Title': get_text_or_none(soup.find('span', class_='a-size-large product-title-word-break')),
            'Brand': get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
            'OS': get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
            'RAM': get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
            'CPU': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
            'Storage': get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
            'Screen Size': get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
            'Resolution': get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
            'CPU Speed': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
            'Model': get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
            'Wireless Provider': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
            'Cellular Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
            'Color': color,
            'Refresh Rate': get_text_or_none(soup.find('tr', class_='a-spacing-small po-refresh_rate').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-refresh_rate') else None),
            'SIM Count' : get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
            'Wireless Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
            'Price': price,
            'Price Before Promotion' : get_text_or_none(soup.find('span', class_='a-price a-text-price').find('span', class_='a-offscreen') if soup.find('span', class_='a-price a-text-price') else None),
            'Rate': get_text_or_none(soup.find('span', class_='a-icon-alt')),
            'Camera': camera_info,
            'IMG': get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
            'Reviews': scrape_amazon_reviews(url)
        }
        driver.quit()
        return data

    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")
        return None

# Scrape product data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_product_data, urls)
    for result in results:
        if result:
            titles.append(result['Title'])
            brands.append(result['Brand'])
            OSs.append(result['OS'])
            rams.append(result['RAM'])
            cpus.append(result['CPU'])
            storages.append(result['Storage'])
            SSs.append(result['Screen Size'])
            resolutions.append(result['Resolution'])
            refresh_rate.append(result['Refresh Rate'])
            cpu_speeds.append(result['CPU Speed'])
            model_names.append(result['Model'])
            wireless_providers.append(result['Wireless Provider'])
            Cellular_technology.append(result['Cellular Technology'])
            WTs.append(result['Wireless Technology'])
            SIMs.append(result['SIM Count'])
            colors.append(result['Color'])
            prices.append(result['Price'])
            beforeprice.append(result['Price Before Promotion'])
            camera.append(result['Camera'])
            rates.append(result['Rate'])
            imgs.append(result['IMG'])
            reviews.append(result['Reviews'])
            Currency.append('EGP')
            Website.append('Amazon EG')

# Create a DataFrame to store the scraped data
df_xiaomi = pd.DataFrame({
    'Title': titles,
    'Brand': brands,
    'OS': OSs,
    'RAM': rams,
    'CPU': cpus,
    'Storage': storages,
    'Screen Size': SSs,
    'Resolution': resolutions,
    'CPU Speed': cpu_speeds,
    'Refresh Rate':refresh_rate,
    'Model': model_names,
    'Wireless Technology' : WTs,
    'SIM Count' : SIMs , 
    'Wireless Provider': wireless_providers,
    'Cellular Technology': Cellular_technology,
    'Camera' : camera,
    'Color': colors,
    'Price': prices,
    'Price Before Promotion':beforeprice,
    'Currancy' : Currency,
    'Rate': rates,
    'URL' : urls,
    'IMG': imgs,
    'Reviews': reviews,
    'Website' : Website
})

df_xiaomi

Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found


,Title,Brand,OS,RAM,CPU,Storage,Screen Size,Resolution,CPU Speed,Refresh Rate,...,Camera,Color,Price,Price Before Promotion,Currancy,Rate,URL,IMG,Reviews,Website
0,Xiaomi Poco F6 5G + 4G LTE 512GB + 12GB Global...,Xiaomi,Powered by Xiaomi HyperOS,None,None,511.99 GB,6.67 Inches,2712 x 1220,3E+3 MHz,120 hertz,...,None,Titanium Global ROM,"EGP20,299.00","EGP21,750.00",EGP,3.3 out of 5 stars,https://www.amazon.eg/-/en/Xiaomi-Version-Unlo...,https://m.media-amazon.com/images/I/510khecAij...,"[{'reviewer_name': 'Marco Medhat', 'review_rat...",Amazon EG
1,Xiaomi Poco F6 5G + 4G LTE 512GB + 12GB Global...,Xiaomi,Android 13.0,None,A-Series,512 GB,6.67 Inches,2712 x 1220,3E+3 MHz,120 hertz,...,None,None,"EGP20,888.00",None,EGP,Previous page,https://www.amazon.eg/-/en/Xiaomi-Global-Versi...,https://m.media-amazon.com/images/I/516KMNsDvr...,[],Amazon EG
2,Xiaomi Poco M6 Pro 4G LTE GSM (256GB + 8GB) 64...,Xiaomi,None,None,None,8 GB,6.67 Inches,None,None,None,...,None,Black Global ROM,"EGP11,999.00",None,EGP,4.5 out of 5 stars,https://www.amazon.eg/-/en/Xiaomi-Triple-Tmobi...,https://m.media-amazon.com/images/I/51GYXsNLNN...,[],Amazon EG
3,Xiaomi Redmi 13C (8Gb Ram + 256Gb Ram) 168 X 7...,Xiaomi,"MIUI 12.5, Android 11.0",8 GB,Snapdragon,256 GB,6.74 Inches,720 x 1600,None,90 Hz,...,None,None,"8,199.",None,EGP,4.0 out of 5 stars,https://www.amazon.eg/-/en/Xiaomi-Redmi-13C-25...,https://m.media-amazon.com/images/I/51O4eb7IRT...,"[{'reviewer_name': 'gamal mansy', 'review_rati...",Amazon EG
4,"Xiaomi Redmi Note 11 Pro 4G Volte 128GB, 8GB 1...",Xiaomi,Android 10.0,8 GB,Snapdragon,8 GB,6.67 Inches,1080 x 2400,2.5 GHz,144 Hz,...,None,Red,"EGP11,999.00",None,EGP,4.5 out of 5 stars,https://www.amazon.eg/-/en/Xiaomi-Redmi-Note-1...,https://m.media-amazon.com/images/I/61vIV+riRE...,"[{'reviewer_name': 'AX', 'review_rating': '2.0...",Amazon EG
5,"Redmi Note 13 Pro 8GB RAM, 256 - Ultra-clear 2...",Xiaomi,Android 13.0,None,None,256 GB,6.67 Inches,None,None,None,...,200MP,Black,"8,199.",None,EGP,4.6 out of 5 stars,https://www.amazon.eg/-/en/Redmi-Note-Midnight...,https://m.media-amazon.com/images/I/51G7tyeELv...,"[{'reviewer_name': 'ممتاز جداا', 'review_ratin...",Amazon EG
6,"MI NOTE 12S 8GB RAM, 256 GB ROM - ONYX BLACK",Xiaomi,android 13.0,8 GB,Snapdragon,256 GB,6.43 Inches,1080 x 2400,None,90 Hz,...,None,None,"8,922.",None,EGP,4.1 out of 5 stars,https://www.amazon.eg/-/en/NOTE-12S-8GB-RAM-25...,https://m.media-amazon.com/images/I/61cKUEmKct...,"[{'reviewer_name': 'Taha', 'review_rating': '1...",Amazon EG
7,"Xiaomi - Redmi Note 11 128GB, 4GB RAM, WiFi, P...",Xiaomi,Android 11.0,4 GB,Snapdragon,4 GB,None,1080 x 2400,None,90 Hz,...,None,None,"3,999.",None,EGP,4.7 out of 5 stars,https://www.amazon.eg/-/en/Xiaomi-Redmi-128GB-...,https://m.media-amazon.com/images/I/71bru4lFzz...,"[{'reviewer_name': 'Esraa Hamdy', 'review_rati...",Amazon EG
8,"Xiaomi Redmi Note 11 Pro 4G (Star Blue, 8GB RA...",Xiaomi,MIUI 12,8 GB,Snapdragon,6 GB,6.67 Inches,1080 x 2400,2.05 GHz,120 GHz,...,None,None,"EGP11,999.00",None,EGP,Previous page,https://www.amazon.eg/-/en/Xiaomi-Redmi-Note-1...,https://m.media-amazon.com/images/I/512C1yLtkX...,[],Amazon EG
9,"Xiaomi Redmi A2+ Dual SIM Smartphone, 3GB RAM,...",Xiaomi,Android 13.0,3 GB,Snapdragon,64 GB,6.52 Inches,2400 x 1080,None,70 Hz,...,None,None,"EGP3,949.00",None,EGP,3.7 out of 5 stars,https://www.amazon.eg/-/en/Xiaomi-Smartphone-H...,https://m.media-amazon.com/images/I/71NA7Z6L7F...,[{'reviewer_name': 'برجاء التأكد من وجود الخرط...,Amazon EG


In [17]:
import pandas as pd
import re
allcolors = [
     "Rose Gold", "Navy Blue","Light Blue","Cobalt Violet","Awesome Graphite","Titanium Black","Wave Green"
    "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink","Carbon Grey",
    "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black","Cobalt Violet",
    "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
    "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White","Moonlight White"
    "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", "Awesome Navy"
    "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", "Awesome Lime","Silver Shadow",
    "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
    "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", "Clear","Navy","Chartreuse",
    "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", "Graphite","Violet",
    "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
]
import pandas as pd
def find_color_in_title(title, allcolors):
    for color in allcolors:
        if color.lower() in title.lower():
            return color
    return None
df_xiaomi['Color'] = df_xiaomi.apply(lambda row: find_color_in_title(row['Title'], allcolors) or row['Color'], axis=1)

In [18]:
df_xiaomi.to_csv('Xiaomi EG.csv',index = False)

Amazon EG OPPO

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Headers for requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

driver = webdriver.Firefox()

urls = []
for page in range(1, 2):
    url = f'https://www.amazon.eg/s?bbn=21832883031&rh=n%3A21832883031%2Cp_89%3AOPPO&language=en'

    driver.get(url)
    page_source = driver.page_source
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')))
    time.sleep(2)
    soup = BeautifulSoup(page_source, 'html.parser')
    url_tags = soup.find_all('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    for url_tag in url_tags:
        urls.append(f"https://www.amazon.eg{url_tag['href']}&language=en_AE")
driver.quit()
urls = list(set(urls))
print(f'{len(urls)} Product URLs retrieved.')

21 Product URLs retrieved.


In [21]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def create_optimized_firefox_options():
    options = Options()
    options.headless = True  # Run browser in headless mode

    # Create a Firefox profile for advanced performance optimizations
    profile = webdriver.FirefoxProfile()

    # Disable images to save bandwidth
    profile.set_preference("permissions.default.image", 2)

    # Disable CSS to speed up page loading (if CSS is not needed)
    profile.set_preference("permissions.default.stylesheet", 2)

    # Disable JavaScript for speed (if JavaScript is not needed for scraping)
    profile.set_preference("javascript.enabled", False)

    # Disable plugins and Flash to reduce resource usage
    profile.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", False)
    profile.set_preference("media.autoplay.default", 0)  # Disable media autoplay

    # Disable WebRTC to minimize network-related overhead
    profile.set_preference("media.peerconnection.enabled", False)

    # Block tracking content and ads
    profile.set_preference("privacy.trackingprotection.enabled", True)
    profile.set_preference("network.http.referer.spoofSource", True)
    profile.set_preference("privacy.trackingprotection.socialtracking.enabled", True)

    # Disable media and video buffering
    profile.set_preference("media.block-autoplay-until-in-foreground", True)
    profile.set_preference("media.autoplay.default", 5)
    profile.set_preference("permissions.default.media", 2)

    # Turn off telemetry to avoid sending data back to Mozilla
    profile.set_preference("datareporting.healthreport.uploadEnabled", False)
    profile.set_preference("toolkit.telemetry.reportingpolicy.firstRun", False)

    # Suppress unnecessary logging to speed up execution
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    options.log.level = "error"  # Minimize logs

    # Attach the profile to Firefox options
    options.profile = profile

    return options  # To run Firefox in headless mode

options = create_optimized_firefox_options()
titles = []
prices = []
rates = []
brands = []
OSs = []
rams = []
cpus = []
storages = []
SSs = []
resolutions = []
refresh_rate= []
model_names = []
wireless_providers = []
colors = []
imgs = []
reviews = []
cpu_speeds = []
Cellular_technology = []
SIMs = []
WTs =[]
camera =[]
beforeprice=[]
Currency = []
Website =[]
def scrape_amazon_reviews(url):
    # Fetch the page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return []
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the reviews section
    reviews_section = soup.find('div', id='cm-cr-dp-review-list')
    if not reviews_section:
        print("Reviews section not found")
        return []
    
    reviews = reviews_section.find_all('div', {'data-hook': 'review'})

    review_data = []
    for review in reviews:
        # Find reviewer name
        reviewer_name = review.find('span', class_='a-profile-name')
        reviewer_name = reviewer_name.text.strip() if reviewer_name else 'N/A'
        
        # Find review rating
        review_rating = review.find('i', {'data-hook': 'review-star-rating'})
        review_rating = review_rating.text.strip() if review_rating else 'N/A'
        
        # Find review title
        review_title = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title.text.strip() if review_title else 'N/A'
        
        # Find review text
        review_text = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text.text.strip() if review_text else 'N/A'
        
        # Find review date
        review_date = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date.text.strip() if review_date else 'N/A'

        review_data.append({
            'reviewer_name': reviewer_name,
            'review_rating': review_rating,
            'review_title': review_title,
            'review_text': review_text,
            'review_date': review_date
        })

    return review_data

def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None

def scrape_product_data(url):
    try:
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-size-large.product-title-word-break')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        camera_pattern = re.compile(r'(\d+MP)\s+MAIN CAMERA')
        camera_pattern2 = re.compile(r'(\d+MP)\s+camera')
        camera_pattern3 = re.compile(r'(\d+ MP)\s+camera')
        camera_pattern4 = re.compile(r'(\d+ MP)\s+MAIN CAMERA')
        camera_info = None
        for li in soup.find_all('li', class_='a-spacing-mini'):
            match = camera_pattern.search(li.text)
            match2 = camera_pattern2.search(li.text)
            match3 = camera_pattern3.search(li.text)
            match4 = camera_pattern4.search(li.text)
            if match:
                camera_info = match.group(1)
                break
            if match2:
                camera_info = match2.group(1)
                break
            if match3:
                camera_info = match3.group(1)
                break
            if match4:
                camera_info = match4.group(1)
                break
        camera_features = None  # Placeholder for camera information
        
        # Find the table with camera features
        camera_features_row = soup.find('table', id='productDetails_techSpec_section_1')
        if camera_features_row:
            # Find the specific row containing "Other camera features"
            camera_features_row2 = camera_features_row.find('th', string='Other camera features')
            
            # Ensure the row exists and extract the camera details
            if camera_features_row2:
                camera_features = camera_features_row2.find_next('td').get_text(strip=True)
                
                # Extract the main camera's MP (assuming 'XXMP main camera' format)
                if 'main camera' in camera_features:
                    # Get the part before "main camera" and extract the megapixels
                    camera_info = camera_features.split('main camera')[0].strip().split()[-1]
        
        color_row = soup.find('tr', class_='a-spacing-small po-color')
        color = None  # Initialize color
        
        if color_row:
            # If the color row exists, find the span inside it and get the text
            color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
        
        elif soup.find('table', id ='productDetails_techSpec_section_1'):
            color_row2 = soup.find('th', string='Color')
            if color_row2:
                color = color_row2.find_next('td').get_text(strip=True)
        
        else:
            # Check for color in div with id='variation_color_name'
            outerSelect = soup.find('div', id='variation_color_name')
            if outerSelect:
                selection_row = outerSelect.find('span', class_='selection')
                if selection_row:
                    color = get_text_or_none(selection_row)
        price = get_text_or_none(soup.find('div', class_='a-section a-spacing-micro').find('span', class_='a-offscreen')
                         if soup.find('div', class_='a-section a-spacing-micro') else None)
        if not price:
            price = get_text_or_none(soup.find('span', class_='a-price-whole'))



        # Scrape data using helper functon
        data = {
            'Title': get_text_or_none(soup.find('span', class_='a-size-large product-title-word-break')),
            'Brand': get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
            'OS': get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
            'RAM': get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
            'CPU': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
            'Storage': get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
            'Screen Size': get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
            'Resolution': get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
            'CPU Speed': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
            'Model': get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
            'Wireless Provider': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
            'Cellular Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
            'Color': color,
            'Refresh Rate': get_text_or_none(soup.find('tr', class_='a-spacing-small po-refresh_rate').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-refresh_rate') else None),
            'SIM Count' : get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
            'Wireless Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
            'Price': price,
            'Price Before Promotion' : get_text_or_none(soup.find('span', class_='a-price a-text-price').find('span', class_='a-offscreen') if soup.find('span', class_='a-price a-text-price') else None),
            'Rate': get_text_or_none(soup.find('span', class_='a-icon-alt')),
            'Camera': camera_info,
            'IMG': get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
            'Reviews': scrape_amazon_reviews(url)
        }
        driver.quit()
        return data

    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")
        return None

# Scrape product data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_product_data, urls)
    for result in results:
        if result:
            titles.append(result['Title'])
            brands.append(result['Brand'])
            OSs.append(result['OS'])
            rams.append(result['RAM'])
            cpus.append(result['CPU'])
            storages.append(result['Storage'])
            SSs.append(result['Screen Size'])
            resolutions.append(result['Resolution'])
            refresh_rate.append(result['Refresh Rate'])
            cpu_speeds.append(result['CPU Speed'])
            model_names.append(result['Model'])
            wireless_providers.append(result['Wireless Provider'])
            Cellular_technology.append(result['Cellular Technology'])
            WTs.append(result['Wireless Technology'])
            SIMs.append(result['SIM Count'])
            colors.append(result['Color'])
            prices.append(result['Price'])
            beforeprice.append(result['Price Before Promotion'])
            camera.append(result['Camera'])
            rates.append(result['Rate'])
            imgs.append(result['IMG'])
            reviews.append(result['Reviews'])
            Currency.append('EGP')
            Website.append('Amazon EG')

# Create a DataFrame to store the scraped data
df_oppo = pd.DataFrame({
    'Title': titles,
    'Brand': brands,
    'OS': OSs,
    'RAM': rams,
    'CPU': cpus,
    'Storage': storages,
    'Screen Size': SSs,
    'Resolution': resolutions,
    'CPU Speed': cpu_speeds,
    'Refresh Rate':refresh_rate,
    'Model': model_names,
    'Wireless Technology' : WTs,
    'SIM Count' : SIMs , 
    'Wireless Provider': wireless_providers,
    'Cellular Technology': Cellular_technology,
    'Camera' : camera,
    'Color': colors,
    'Price': prices,
    'Price Before Promotion':beforeprice,
    'Currancy' : Currency,
    'Rate': rates,
    'URL' : urls,
    'IMG': imgs,
    'Reviews': reviews,
    'Website' : Website
})

df_oppo

Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found


,Title,Brand,OS,RAM,CPU,Storage,Screen Size,Resolution,CPU Speed,Refresh Rate,...,Camera,Color,Price,Price Before Promotion,Currancy,Rate,URL,IMG,Reviews,Website
0,OPPO A18 Dual SIM 4GB RAM+64GB ROM (6.56 Inche...,OPPO,Android 13.0,4 GB,Snapdragon,64 GB,6.56 Inches,720 x 1612,None,90 Hz,...,None,Glowing Blue,"3,999.",None,EGP,4.0 out of 5 stars,https://www.amazon.eg/-/en/OPPO-Dual-64GB-6-56...,https://m.media-amazon.com/images/I/81r7k-Iocl...,[],Amazon EG
1,OPPO Reno 6 Dual Sim 8gb Ram 128gb 4G Lte Phon...,OPPO,android,8 GB,Snapdragon,128 GB,6.4 Inches,1080 x 2400,1.8 GHz,70 Hz,...,None,None,"EGP13,499.00",None,EGP,4.2 out of 5 stars,https://www.amazon.eg/-/en/OPPO-Reno-128gb-Pho...,https://m.media-amazon.com/images/I/61bt1qv5jZ...,"[{'reviewer_name': 'ahmed elshishtawy', 'revie...",Amazon EG
2,Oppo Reno11 5G 12-256 Wave Green,OPPO,Android 11.0,12 GB,Snapdragon,256 GB,6.55 Inches,None,None,None,...,None,Wave Green,"EGP16,490.00",None,EGP,1.0 out of 5 stars,https://www.amazon.eg/-/en/Oppo-Reno11-12-256-...,https://m.media-amazon.com/images/I/61D1fQ4EA3...,[{'reviewer_name': 'Mohammed Ali Abdelnabi Att...,Amazon EG
3,"Oppo Reno 12F 4G, 8GB Ram Snapdragon 685, 256G...",None,None,None,None,None,None,None,None,None,...,None,None,"EGP10,599.00",None,EGP,1.0 out of 5 stars,https://www.amazon.eg/-/en/Oppo-Snapdragon-256...,https://m.media-amazon.com/images/I/51+PGGwPBm...,"[{'reviewer_name': 'Ahmed Gamal', 'review_rati...",Amazon EG
4,Oppo A79 5G Dual-SIM 256GB ROM + 8GB RAM (Only...,OPPO,Android 13.0,8 GB,None,8 GB,6.4 Inches,None,2.2 GHz,None,...,None,Mystery Black,"EGP9,990.00",None,EGP,3.5 out of 5 stars,https://www.amazon.eg/-/en/Dual-SIM-Factory-Un...,https://m.media-amazon.com/images/I/71Fbf9Obn1...,[],Amazon EG
5,"OPPO Reno 5A 5G Single Nano Sim +esim A1030P, ...",OPPO,Android 11.0,6 GB,Snapdragon,128 GB,None,1080 x 2340,1.8 GHz,120 Hz,...,None,None,"3,999.",None,EGP,5.0 out of 5 stars,https://www.amazon.eg/-/en/OPPO-Display-Waterp...,https://m.media-amazon.com/images/I/51OwigwPAw...,"[{'reviewer_name': 'ميدو', 'review_rating': '5...",Amazon EG
6,Oppo A58 Dual SIM 6GB RAM + 128GB ROM (4G) (Bl...,OPPO,Android 13.0,6 GB,Snapdragon,128 GB,6.72 Inches,2160 x 1080,None,60 Hz,...,None,black,"EGP7,650.00","EGP8,449.00",EGP,4.0 out of 5 stars,https://www.amazon.eg/-/en/Oppo-A58-Dual-128GB...,https://m.media-amazon.com/images/I/81gpCfWufk...,"[{'reviewer_name': 'Ashraf omar', 'review_rati...",Amazon EG
7,OPPO Reno 6 5G Dual SIM Stellar Black 8GB RAM ...,OPPO,Android 11.0,8 GB,Snapdragon,128 GB,6.4 Inches,1080 x 2400,2 GHz,90 Hz,...,None,Stellar black,"8,199.",None,EGP,3.4 out of 5 stars,https://www.amazon.eg/-/en/OPPO-Reno-Stellar-B...,https://m.media-amazon.com/images/I/51jGptxULl...,"[{'reviewer_name': 'Ahmed Mohamed', 'review_ra...",Amazon EG
8,OPPO Reno 12F 5G (256GB - 12GB) - Amber Orange,OPPO,None,None,None,12 GB,None,None,None,None,...,None,None,"EGP12,760.00","EGP13,349.00",EGP,2.4 out of 5 stars,https://www.amazon.eg/-/en/OPPO-Reno-12F-5G-25...,https://m.media-amazon.com/images/I/518W2EY7-7...,"[{'reviewer_name': 'user-MHWXUG1', 'review_rat...",Amazon EG
9,"Oppo Reno 12F 4G, 8GB Ram Snapdragon 685, 256G...",None,None,None,None,None,None,None,None,None,...,None,None,"EGP10,475.00",None,EGP,5.0 out of 5 stars,https://www.amazon.eg/-/en/Oppo-Snapdragon-256...,https://m.media-amazon.com/images/I/417+9I-0j8...,"[{'reviewer_name': 'شيماء شعبان محمد', 'review...",Amazon EG


In [22]:
import pandas as pd
import re
allcolors = [
     "Rose Gold", "Navy Blue","Light Blue","Cobalt Violet","Awesome Graphite","Titanium Black","Wave Green"
    "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink","Carbon Grey",
    "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black","Cobalt Violet",
    "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
    "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White","Moonlight White"
    "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", "Awesome Navy"
    "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", "Awesome Lime","Silver Shadow",
    "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
    "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", "Clear","Navy","Chartreuse",
    "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", "Graphite","Violet",
    "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
]
import pandas as pd
def find_color_in_title(title, allcolors):
    for color in allcolors:
        if color.lower() in title.lower():
            return color
    return None
df_oppo['Color'] = df_oppo.apply(lambda row: find_color_in_title(row['Title'], allcolors) or row['Color'], axis=1)

In [23]:
df_oppo.to_csv('OPPO EG.csv' , index = False)

Amazon EG realme

In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Headers for requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

driver = webdriver.Firefox()

urls = []
for page in range(1, 3):
    url = f'https://www.amazon.eg/s?i=electronics&bbn=21832883031&rh=n%3A21832883031%2Cp_89%3Arealme&page={page}&language=en_AE'

    driver.get(url)
    page_source = driver.page_source
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')))
    time.sleep(2)
    soup = BeautifulSoup(page_source, 'html.parser')
    url_tags = soup.find_all('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    for url_tag in url_tags:
        urls.append(f"https://www.amazon.eg{url_tag['href']}&language=en_AE")
driver.quit()
urls = list(set(urls))
print(f'{len(urls)} Product URLs retrieved.')

42 Product URLs retrieved.


In [25]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def create_optimized_firefox_options():
    options = Options()
    options.headless = True  # Run browser in headless mode

    # Create a Firefox profile for advanced performance optimizations
    profile = webdriver.FirefoxProfile()

    # Disable images to save bandwidth
    profile.set_preference("permissions.default.image", 2)

    # Disable CSS to speed up page loading (if CSS is not needed)
    profile.set_preference("permissions.default.stylesheet", 2)

    # Disable JavaScript for speed (if JavaScript is not needed for scraping)
    profile.set_preference("javascript.enabled", False)

    # Disable plugins and Flash to reduce resource usage
    profile.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", False)
    profile.set_preference("media.autoplay.default", 0)  # Disable media autoplay

    # Disable WebRTC to minimize network-related overhead
    profile.set_preference("media.peerconnection.enabled", False)

    # Block tracking content and ads
    profile.set_preference("privacy.trackingprotection.enabled", True)
    profile.set_preference("network.http.referer.spoofSource", True)
    profile.set_preference("privacy.trackingprotection.socialtracking.enabled", True)

    # Disable media and video buffering
    profile.set_preference("media.block-autoplay-until-in-foreground", True)
    profile.set_preference("media.autoplay.default", 5)
    profile.set_preference("permissions.default.media", 2)

    # Turn off telemetry to avoid sending data back to Mozilla
    profile.set_preference("datareporting.healthreport.uploadEnabled", False)
    profile.set_preference("toolkit.telemetry.reportingpolicy.firstRun", False)

    # Suppress unnecessary logging to speed up execution
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    options.log.level = "error"  # Minimize logs

    # Attach the profile to Firefox options
    options.profile = profile

    return options  # To run Firefox in headless mode

options = create_optimized_firefox_options()
titles = []
prices = []
rates = []
brands = []
OSs = []
rams = []
cpus = []
storages = []
SSs = []
resolutions = []
refresh_rate= []
model_names = []
wireless_providers = []
colors = []
imgs = []
reviews = []
cpu_speeds = []
Cellular_technology = []
SIMs = []
WTs =[]
camera =[]
beforeprice=[]
Currency = []
Website =[]
def scrape_amazon_reviews(url):
    # Fetch the page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return []
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the reviews section
    reviews_section = soup.find('div', id='cm-cr-dp-review-list')
    if not reviews_section:
        print("Reviews section not found")
        return []
    
    reviews = reviews_section.find_all('div', {'data-hook': 'review'})

    review_data = []
    for review in reviews:
        # Find reviewer name
        reviewer_name = review.find('span', class_='a-profile-name')
        reviewer_name = reviewer_name.text.strip() if reviewer_name else 'N/A'
        
        # Find review rating
        review_rating = review.find('i', {'data-hook': 'review-star-rating'})
        review_rating = review_rating.text.strip() if review_rating else 'N/A'
        
        # Find review title
        review_title = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title.text.strip() if review_title else 'N/A'
        
        # Find review text
        review_text = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text.text.strip() if review_text else 'N/A'
        
        # Find review date
        review_date = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date.text.strip() if review_date else 'N/A'

        review_data.append({
            'reviewer_name': reviewer_name,
            'review_rating': review_rating,
            'review_title': review_title,
            'review_text': review_text,
            'review_date': review_date
        })

    return review_data

def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None

def scrape_product_data(url):
    try:
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-size-large.product-title-word-break')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        camera_pattern = re.compile(r'(\d+MP)\s+MAIN CAMERA')
        camera_pattern2 = re.compile(r'(\d+MP)\s+camera')
        camera_pattern3 = re.compile(r'(\d+ MP)\s+camera')
        camera_pattern4 = re.compile(r'(\d+ MP)\s+MAIN CAMERA')
        camera_info = None
        for li in soup.find_all('li', class_='a-spacing-mini'):
            match = camera_pattern.search(li.text)
            match2 = camera_pattern2.search(li.text)
            match3 = camera_pattern3.search(li.text)
            match4 = camera_pattern4.search(li.text)
            if match:
                camera_info = match.group(1)
                break
            if match2:
                camera_info = match2.group(1)
                break
            if match3:
                camera_info = match3.group(1)
                break
            if match4:
                camera_info = match4.group(1)
                break
        camera_features = None  # Placeholder for camera information
        
        # Find the table with camera features
        camera_features_row = soup.find('table', id='productDetails_techSpec_section_1')
        if camera_features_row:
            # Find the specific row containing "Other camera features"
            camera_features_row2 = camera_features_row.find('th', string='Other camera features')
            
            # Ensure the row exists and extract the camera details
            if camera_features_row2:
                camera_features = camera_features_row2.find_next('td').get_text(strip=True)
                
                # Extract the main camera's MP (assuming 'XXMP main camera' format)
                if 'main camera' in camera_features:
                    # Get the part before "main camera" and extract the megapixels
                    camera_info = camera_features.split('main camera')[0].strip().split()[-1]
        
        color_row = soup.find('tr', class_='a-spacing-small po-color')
        color = None  # Initialize color
        
        if color_row:
            # If the color row exists, find the span inside it and get the text
            color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
        
        elif soup.find('table', id ='productDetails_techSpec_section_1'):
            color_row2 = soup.find('th', string='Color')
            if color_row2:
                color = color_row2.find_next('td').get_text(strip=True)
        
        else:
            # Check for color in div with id='variation_color_name'
            outerSelect = soup.find('div', id='variation_color_name')
            if outerSelect:
                selection_row = outerSelect.find('span', class_='selection')
                if selection_row:
                    color = get_text_or_none(selection_row)
        price = get_text_or_none(soup.find('div', class_='a-section a-spacing-micro').find('span', class_='a-offscreen')
                         if soup.find('div', class_='a-section a-spacing-micro') else None)
        if not price:
            price = get_text_or_none(soup.find('span', class_='a-price-whole'))



        # Scrape data using helper functon
        data = {
            'Title': get_text_or_none(soup.find('span', class_='a-size-large product-title-word-break')),
            'Brand': get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
            'OS': get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
            'RAM': get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
            'CPU': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
            'Storage': get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
            'Screen Size': get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
            'Resolution': get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
            'CPU Speed': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
            'Model': get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
            'Wireless Provider': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
            'Cellular Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
            'Color': color,
            'Refresh Rate': get_text_or_none(soup.find('tr', class_='a-spacing-small po-refresh_rate').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-refresh_rate') else None),
            'SIM Count' : get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
            'Wireless Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
            'Price': price,
            'Price Before Promotion' : get_text_or_none(soup.find('span', class_='a-price a-text-price').find('span', class_='a-offscreen') if soup.find('span', class_='a-price a-text-price') else None),
            'Rate': get_text_or_none(soup.find('span', class_='a-icon-alt')),
            'Camera': camera_info,
            'IMG': get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
            'Reviews': scrape_amazon_reviews(url)
        }
        driver.quit()
        return data

    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")
        return None

# Scrape product data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_product_data, urls)
    for result in results:
        if result:
            titles.append(result['Title'])
            brands.append(result['Brand'])
            OSs.append(result['OS'])
            rams.append(result['RAM'])
            cpus.append(result['CPU'])
            storages.append(result['Storage'])
            SSs.append(result['Screen Size'])
            resolutions.append(result['Resolution'])
            refresh_rate.append(result['Refresh Rate'])
            cpu_speeds.append(result['CPU Speed'])
            model_names.append(result['Model'])
            wireless_providers.append(result['Wireless Provider'])
            Cellular_technology.append(result['Cellular Technology'])
            WTs.append(result['Wireless Technology'])
            SIMs.append(result['SIM Count'])
            colors.append(result['Color'])
            prices.append(result['Price'])
            beforeprice.append(result['Price Before Promotion'])
            camera.append(result['Camera'])
            rates.append(result['Rate'])
            imgs.append(result['IMG'])
            reviews.append(result['Reviews'])
            Currency.append('EGP')
            Website.append('Amazon EG')

# Create a DataFrame to store the scraped data
df_realme = pd.DataFrame({
    'Title': titles,
    'Brand': brands,
    'OS': OSs,
    'RAM': rams,
    'CPU': cpus,
    'Storage': storages,
    'Screen Size': SSs,
    'Resolution': resolutions,
    'CPU Speed': cpu_speeds,
    'Refresh Rate':refresh_rate,
    'Model': model_names,
    'Wireless Technology' : WTs,
    'SIM Count' : SIMs , 
    'Wireless Provider': wireless_providers,
    'Cellular Technology': Cellular_technology,
    'Camera' : camera,
    'Color': colors,
    'Price': prices,
    'Price Before Promotion':beforeprice,
    'Currancy' : Currency,
    'Rate': rates,
    'URL' : urls,
    'IMG': imgs,
    'Reviews': reviews,
    'Website' : Website
})

df_realme

Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found


,Title,Brand,OS,RAM,CPU,Storage,Screen Size,Resolution,CPU Speed,Refresh Rate,...,Camera,Color,Price,Price Before Promotion,Currancy,Rate,URL,IMG,Reviews,Website
0,"realme 12 Pro 5G (Submarine Blue, 12GB RAM, 51...",realme,realme UI 4.0 || Based on Android 13,8 GB,Snapdragon,512 GB,6.7 Inches,1080 x 2400,None,60 Hz,...,None,Submarine Blue,"EGP16,469.00",None,EGP,4.2 out of 5 stars,https://www.amazon.eg/-/en/realme-Submarine-Fl...,https://m.media-amazon.com/images/I/71emLVOIWd...,"[{'reviewer_name': 'Mohamad Al Houssiny', 'rev...",Amazon EG
1,realme Realme C51 Dual-SIM 256GB ROM + 6GB RAM...,realme,Android 13.0,6 GB,Snapdragon,6 GB,6.5 Inches,1080 x 2400,None,None,...,None,None,"8,199.",None,EGP,2.9 out of 5 stars,https://www.amazon.eg/-/en/realme-Realme-Dual-...,https://m.media-amazon.com/images/I/51JifQ3po9...,[],Amazon EG
2,"Realme c25y dual sim, 6.5-inch, 128gb rom, 4gb...",realme,Android 11.0,128 GB,Snapdragon,128 GB,6.5 Inches,720 x 1600,1.8 GHz,120 Hz,...,None,None,"6,499.",None,EGP,3.8 out of 5 stars,https://www.amazon.eg/-/en/Realme-c25y-dual-6-...,https://m.media-amazon.com/images/I/61JwZOfHAO...,"[{'reviewer_name': 'Mohamed Ramadan', 'review_...",Amazon EG
3,Realme Note 50 4-128 Black,realme,Android 10.0,4 GB,Snapdragon,128 GB,6.4 Inches,None,None,None,...,None,Midnight Black,"3,299.",None,EGP,1.0 out of 5 stars,https://www.amazon.eg/-/en/Realme-Note-50-4-12...,https://m.media-amazon.com/images/I/51vbgjrzZ4...,"[{'reviewer_name': 'رشا حسن', 'review_rating':...",Amazon EG
4,"realme 7i 128 GB, 8GB RAM 4G Lte Dual Sim Phon...",realme,Android 10.0,8 GB,Snapdragon,128 GB,6.5 Inches,720 x 1600,2.84 GHz,90 Hz,...,None,Blue,"EGP7,399.00",None,EGP,5.0 out of 5 stars,https://www.amazon.eg/-/en/realme-7i-Dual-Phon...,https://m.media-amazon.com/images/I/51y0vE4L85...,"[{'reviewer_name': 'MØHÄMMËD ĒSSĀ', 'review_ra...",Amazon EG
5,"realme c65 Smartphone 6+128 GB, 50MP AI Camera...",realme,Android,None,None,128 GB,6.4 Inches,None,None,None,...,None,gold,"EGP6,490.00","EGP6,750.00",EGP,3.9 out of 5 stars,https://www.amazon.eg/-/en/realme-Smartphone-C...,https://m.media-amazon.com/images/I/71nva4btzu...,[],Amazon EG
6,"Realme C33 4GB RAM, 128GB Storage - Night Sea,...",realme,Android 12.0,4 GB,Snapdragon,128 GB,6.5 Inches,720 x 1600,1.8 GHz,60 Hz,...,None,black,"EGP6,999.00",None,EGP,3.0 out of 5 stars,https://www.amazon.eg/-/en/Realme-C33-4GB-128G...,https://m.media-amazon.com/images/I/51w5z3wOpT...,[{'reviewer_name': 'سيئ جدا و بيعلق من اول اسب...,Amazon EG
7,Realme Note 50 DUAL SIM 128GB ROM + 4GB RAM (G...,realme,Android,4 GB,Snapdragon,128 GB,6.74 Inches,720 x 1600,1.8 GHz,None,...,None,None,"8,199.",None,EGP,4.1 out of 5 stars,https://www.amazon.eg/-/en/Realme-128GB-Factor...,https://m.media-amazon.com/images/I/71vWEdLvs-...,[],Amazon EG
8,"realme C21Y Dual Sim 64GB 4GB RAM, 4G Lte Phon...",realme,Android 10.0,4 GB,Snapdragon,4 GB,6.5 Inches,720 x 1600,None,60 Hz,...,None,None,"3,299.",None,EGP,4.3 out of 5 stars,https://www.amazon.eg/-/en/realme-C21Y-Phone-6...,https://m.media-amazon.com/images/I/61jAfCRJq2...,"[{'reviewer_name': 'Bassem Dolf Faleh', 'revie...",Amazon EG
9,"Realme 12 5G Android Smartphone, Dual SIM Mobi...",realme,Android,None,None,512 GB,6.72 Inches,None,None,None,...,None,Woodland Green,"EGP12,550.00",None,EGP,Previous page,https://www.amazon.eg/-/en/Realme-Android-Smar...,https://m.media-amazon.com/images/I/81dA4WginX...,[],Amazon EG


In [26]:
import pandas as pd
import re
allcolors = [
     "Rose Gold", "Navy Blue","Light Blue","Cobalt Violet","Awesome Graphite","Titanium Black","Wave Green"
    "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink","Carbon Grey",
    "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black","Cobalt Violet",
    "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
    "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White","Moonlight White"
    "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", "Awesome Navy"
    "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", "Awesome Lime","Silver Shadow",
    "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
    "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", "Clear","Navy","Chartreuse",
    "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", "Graphite","Violet",
    "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
]
import pandas as pd
def find_color_in_title(title, allcolors):
    for color in allcolors:
        if color.lower() in title.lower():
            return color
    return None
df_realme['Color'] = df_realme.apply(lambda row: find_color_in_title(row['Title'], allcolors) or row['Color'], axis=1)

In [27]:
df_realme.to_csv('Realme EG.csv', index = False)

Amazon NL HUAWEI

In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Headers for requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

driver = webdriver.Firefox()

urls = []
for page in range(1, 2):
    url = f'https://www.amazon.eg/s?k=Honor&i=electronics&rh=n%3A21832883031%2Cp_123%3A1500397&dc&language=en&ds=v1%3AS8VkXDZex2xhwNjjjRX4DkhdjhX29x15B50gbNB4ApM'

    driver.get(url)
    page_source = driver.page_source
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal')))
    time.sleep(2)
    soup = BeautifulSoup(page_source, 'html.parser')
    url_tags = soup.find_all('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    for url_tag in url_tags:
        urls.append(f"https://www.amazon.eg{url_tag['href']}&language=en_AE")
driver.quit()
urls = list(set(urls))
print(f'{len(urls)} Product URLs retrieved.')

19 Product URLs retrieved.


In [31]:
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
def create_optimized_firefox_options():
    options = Options()
    options.headless = True  # Run browser in headless mode

    # Create a Firefox profile for advanced performance optimizations
    profile = webdriver.FirefoxProfile()

    # Disable images to save bandwidth
    profile.set_preference("permissions.default.image", 2)

    # Disable CSS to speed up page loading (if CSS is not needed)
    profile.set_preference("permissions.default.stylesheet", 2)

    # Disable JavaScript for speed (if JavaScript is not needed for scraping)
    profile.set_preference("javascript.enabled", False)

    # Disable plugins and Flash to reduce resource usage
    profile.set_preference("dom.ipc.plugins.enabled.libflashplayer.so", False)
    profile.set_preference("media.autoplay.default", 0)  # Disable media autoplay

    # Disable WebRTC to minimize network-related overhead
    profile.set_preference("media.peerconnection.enabled", False)

    # Block tracking content and ads
    profile.set_preference("privacy.trackingprotection.enabled", True)
    profile.set_preference("network.http.referer.spoofSource", True)
    profile.set_preference("privacy.trackingprotection.socialtracking.enabled", True)

    # Disable media and video buffering
    profile.set_preference("media.block-autoplay-until-in-foreground", True)
    profile.set_preference("media.autoplay.default", 5)
    profile.set_preference("permissions.default.media", 2)

    # Turn off telemetry to avoid sending data back to Mozilla
    profile.set_preference("datareporting.healthreport.uploadEnabled", False)
    profile.set_preference("toolkit.telemetry.reportingpolicy.firstRun", False)

    # Suppress unnecessary logging to speed up execution
    profile.set_preference("browser.download.manager.showWhenStarting", False)
    options.log.level = "error"  # Minimize logs

    # Attach the profile to Firefox options
    options.profile = profile

    return options  # To run Firefox in headless mode

options = create_optimized_firefox_options()
titles = []
prices = []
rates = []
brands = []
OSs = []
rams = []
cpus = []
storages = []
SSs = []
resolutions = []
refresh_rate= []
model_names = []
wireless_providers = []
colors = []
imgs = []
reviews = []
cpu_speeds = []
Cellular_technology = []
SIMs = []
WTs =[]
camera =[]
beforeprice=[]
Currency = []
Website =[]
def scrape_amazon_reviews(url):
    # Fetch the page content
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return []
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the reviews section
    reviews_section = soup.find('div', id='cm-cr-dp-review-list')
    if not reviews_section:
        print("Reviews section not found")
        return []
    
    reviews = reviews_section.find_all('div', {'data-hook': 'review'})

    review_data = []
    for review in reviews:
        # Find reviewer name
        reviewer_name = review.find('span', class_='a-profile-name')
        reviewer_name = reviewer_name.text.strip() if reviewer_name else 'N/A'
        
        # Find review rating
        review_rating = review.find('i', {'data-hook': 'review-star-rating'})
        review_rating = review_rating.text.strip() if review_rating else 'N/A'
        
        # Find review title
        review_title = review.find('a', {'data-hook': 'review-title'})
        review_title = review_title.text.strip() if review_title else 'N/A'
        
        # Find review text
        review_text = review.find('span', {'data-hook': 'review-body'})
        review_text = review_text.text.strip() if review_text else 'N/A'
        
        # Find review date
        review_date = review.find('span', {'data-hook': 'review-date'})
        review_date = review_date.text.strip() if review_date else 'N/A'

        review_data.append({
            'reviewer_name': reviewer_name,
            'review_rating': review_rating,
            'review_title': review_title,
            'review_text': review_text,
            'review_date': review_date
        })

    return review_data

def get_text_or_none(tag, attribute_name=None):
    if tag:
        if attribute_name:
            return tag.get(attribute_name, None)
        return tag.get_text(strip=True)
    return None

def scrape_product_data(url):
    try:
        driver = webdriver.Firefox(options=options)
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-size-large.product-title-word-break')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        camera_pattern = re.compile(r'(\d+MP)\s+MAIN CAMERA')
        camera_pattern2 = re.compile(r'(\d+MP)\s+camera')
        camera_pattern3 = re.compile(r'(\d+ MP)\s+camera')
        camera_pattern4 = re.compile(r'(\d+ MP)\s+MAIN CAMERA')
        camera_info = None
        for li in soup.find_all('li', class_='a-spacing-mini'):
            match = camera_pattern.search(li.text)
            match2 = camera_pattern2.search(li.text)
            match3 = camera_pattern3.search(li.text)
            match4 = camera_pattern4.search(li.text)
            if match:
                camera_info = match.group(1)
                break
            if match2:
                camera_info = match2.group(1)
                break
            if match3:
                camera_info = match3.group(1)
                break
            if match4:
                camera_info = match4.group(1)
                break
        camera_features = None  # Placeholder for camera information
        
        # Find the table with camera features
        camera_features_row = soup.find('table', id='productDetails_techSpec_section_1')
        if camera_features_row:
            # Find the specific row containing "Other camera features"
            camera_features_row2 = camera_features_row.find('th', string='Other camera features')
            
            # Ensure the row exists and extract the camera details
            if camera_features_row2:
                camera_features = camera_features_row2.find_next('td').get_text(strip=True)
                
                # Extract the main camera's MP (assuming 'XXMP main camera' format)
                if 'main camera' in camera_features:
                    # Get the part before "main camera" and extract the megapixels
                    camera_info = camera_features.split('main camera')[0].strip().split()[-1]
        
        color_row = soup.find('tr', class_='a-spacing-small po-color')
        color = None  # Initialize color
        
        if color_row:
            # If the color row exists, find the span inside it and get the text
            color = get_text_or_none(color_row.find('span', class_='a-size-base po-break-word'))
        
        elif soup.find('table', id ='productDetails_techSpec_section_1'):
            color_row2 = soup.find('th', string='Color')
            if color_row2:
                color = color_row2.find_next('td').get_text(strip=True)
        
        else:
            # Check for color in div with id='variation_color_name'
            outerSelect = soup.find('div', id='variation_color_name')
            if outerSelect:
                selection_row = outerSelect.find('span', class_='selection')
                if selection_row:
                    color = get_text_or_none(selection_row)
        price = get_text_or_none(soup.find('div', class_='a-section a-spacing-micro').find('span', class_='a-offscreen')
                         if soup.find('div', class_='a-section a-spacing-micro') else None)
        if not price:
            price = get_text_or_none(soup.find('span', class_='a-price-whole'))



        # Scrape data using helper functon
        data = {
            'Title': get_text_or_none(soup.find('span', class_='a-size-large product-title-word-break')),
            'Brand': get_text_or_none(soup.find('tr', class_='a-spacing-small po-brand').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-brand') else None),
            'OS': get_text_or_none(soup.find('tr', class_='a-spacing-small po-operating_system').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-operating_system') else None),
            'RAM': get_text_or_none(soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-ram_memory.installed_size') else None),
            'CPU': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.family').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.family') else None),
            'Storage': get_text_or_none(soup.find('tr', class_='a-spacing-small po-memory_storage_capacity').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-memory_storage_capacity') else None),
            'Screen Size': get_text_or_none(soup.find('tr', class_='a-spacing-small po-display.size').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-display.size') else None),
            'Resolution': get_text_or_none(soup.find('tr', class_='a-spacing-small po-resolution').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-resolution') else None),
            'CPU Speed': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cpu_model.speed').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cpu_model.speed') else None),
            'Model': get_text_or_none(soup.find('tr', class_='a-spacing-small po-model_name').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-model_name') else None),
            'Wireless Provider': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_provider').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_provider') else None),
            'Cellular Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-cellular_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-cellular_technology') else None),
            'Color': color,
            'Refresh Rate': get_text_or_none(soup.find('tr', class_='a-spacing-small po-refresh_rate').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-refresh_rate') else None),
            'SIM Count' : get_text_or_none(soup.find('tr', class_='a-spacing-small po-sim_card_slot_count').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-sim_card_slot_count') else None),
            'Wireless Technology': get_text_or_none(soup.find('tr', class_='a-spacing-small po-wireless_network_technology').find('span', class_='a-size-base po-break-word') if soup.find('tr', class_='a-spacing-small po-wireless_network_technology') else None),
            'Price': price,
            'Price Before Promotion' : get_text_or_none(soup.find('span', class_='a-price a-text-price').find('span', class_='a-offscreen') if soup.find('span', class_='a-price a-text-price') else None),
            'Rate': get_text_or_none(soup.find('span', class_='a-icon-alt')),
            'Camera': camera_info,
            'IMG': get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'data-old-hires') or get_text_or_none(soup.find('img', class_='a-dynamic-image'), 'src'),
            'Reviews': scrape_amazon_reviews(url)
        }
        driver.quit()
        return data

    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")
        return None

# Scrape product data concurrently
with ThreadPoolExecutor(max_workers=5) as executor:
    results = executor.map(scrape_product_data, urls)
    for result in results:
        if result:
            titles.append(result['Title'])
            brands.append(result['Brand'])
            OSs.append(result['OS'])
            rams.append(result['RAM'])
            cpus.append(result['CPU'])
            storages.append(result['Storage'])
            SSs.append(result['Screen Size'])
            resolutions.append(result['Resolution'])
            refresh_rate.append(result['Refresh Rate'])
            cpu_speeds.append(result['CPU Speed'])
            model_names.append(result['Model'])
            wireless_providers.append(result['Wireless Provider'])
            Cellular_technology.append(result['Cellular Technology'])
            WTs.append(result['Wireless Technology'])
            SIMs.append(result['SIM Count'])
            colors.append(result['Color'])
            prices.append(result['Price'])
            beforeprice.append(result['Price Before Promotion'])
            camera.append(result['Camera'])
            rates.append(result['Rate'])
            imgs.append(result['IMG'])
            reviews.append(result['Reviews'])
            Currency.append('EGP')
            Website.append('Amazon EG')

# Create a DataFrame to store the scraped data
df_honor = pd.DataFrame({
    'Title': titles,
    'Brand': brands,
    'OS': OSs,
    'RAM': rams,
    'CPU': cpus,
    'Storage': storages,
    'Screen Size': SSs,
    'Resolution': resolutions,
    'CPU Speed': cpu_speeds,
    'Refresh Rate':refresh_rate,
    'Model': model_names,
    'Wireless Technology' : WTs,
    'SIM Count' : SIMs , 
    'Wireless Provider': wireless_providers,
    'Cellular Technology': Cellular_technology,
    'Camera' : camera,
    'Color': colors,
    'Price': prices,
    'Price Before Promotion':beforeprice,
    'Currancy' : Currency,
    'Rate': rates,
    'URL' : urls,
    'IMG': imgs,
    'Reviews': reviews,
    'Website' : Website
})

df_honor

Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found
Reviews section not found


,Title,Brand,OS,RAM,CPU,Storage,Screen Size,Resolution,CPU Speed,Refresh Rate,...,Camera,Color,Price,Price Before Promotion,Currancy,Rate,URL,IMG,Reviews,Website
0,HONOR Magic5 Pro 12+512 Meadow Green,HONOR,Android 10.0,12 GB,Snapdragon,12 GB,6.81 Inches,3840 x 2160,None,120 Hz,...,None,Green,"EGP37,950.00",None,EGP,3.8 out of 5 stars,https://www.amazon.eg/HONOR-Magic5-Pro-Meadow-...,https://m.media-amazon.com/images/I/61gf3WPvl7...,[],Amazon EG
1,Honor X7b 6-256 GREEN,HONOR,Android 13.0,6 GB,Snapdragon,256 GB,None,None,None,120 Hz,...,None,Emerald Green,"EGP8,888.00",None,EGP,Previous page,https://www.amazon.eg/-/en/HONOR-6936520833290...,https://m.media-amazon.com/images/I/51ykLhmalQ...,[],Amazon EG
2,"Honor X9a Dual SIM, 8GB Up to 13GB RAM, 256GB ...",HONOR,Android 12.0,8 GB,Snapdragon,256 GB,6.67 Inches,1080 x 2400,120 Hz,80 Hz,...,None,Titanium Silver,"12,764.",None,EGP,3.8 out of 5 stars,https://www.amazon.eg/-/en/Honor-X9a-Dual-13GB...,https://m.media-amazon.com/images/I/61-b0FNJs0...,"[{'reviewer_name': 'Abdelhameid Hamada', 'revi...",Amazon EG
3,HONOR X9b Dual SIM 12GB RAM+256GB ROM (6.78 In...,HONOR,Android 13.0,12 GB,Snapdragon,256 GB,6.78 Inches,None,None,120 Hz,...,None,None,"EGP27,002.38",None,EGP,1.0 out of 5 stars,https://www.amazon.eg/-/en/HONOR-256GB-Inches-...,https://m.media-amazon.com/images/I/51KpKeyZyj...,"[{'reviewer_name': 'مصطفى أحمد سيف', 'review_r...",Amazon EG
4,Honor 90 Lite Dual SIM 8GB RAM+256GB ROM (6.7 ...,HONOR,Android 11.0,8 GB,Snapdragon,256 GB,6.7 Inches,None,None,120 Hz,...,None,None,"EGP11,299.00",None,EGP,Previous page,https://www.amazon.eg/-/en/Honor-Lite-Dual-256...,https://m.media-amazon.com/images/I/61kzigWh3X...,[],Amazon EG
5,HONOR X9b Dual SIM 12GB RAM+256GB ROM (6.78 In...,HONOR,Android 13.0,12 GB,Snapdragon,256 GB,6.78 Inches,None,None,120 Hz,...,None,None,"EGP27,002.38",None,EGP,1.0 out of 5 stars,https://www.amazon.eg/-/en/HONOR-256GB-Inches-...,https://m.media-amazon.com/images/I/61JpJuXpNd...,"[{'reviewer_name': 'احمد ثروت', 'review_rating...",Amazon EG
6,HONOR X9b Dual SIM 12GB RAM+256GB ROM (6.78 In...,HONOR,Android 13.0,12 GB,Snapdragon,256 GB,6.78 Inches,None,None,120 Hz,...,None,None,"EGP27,002.38",None,EGP,Previous page,https://www.amazon.eg/-/en/HONOR-Inches-Emeral...,https://m.media-amazon.com/images/I/5154R0JDaP...,[],Amazon EG
7,Honor X7b 6-256 BLACK,HONOR,Android 13.0,6 GB,Snapdragon,256 GB,6.8 Inches,None,None,120 Hz,...,None,None,"EGP8,600.00",None,EGP,5.0 out of 5 stars,https://www.amazon.eg/-/en/HONOR-6936520833283...,https://m.media-amazon.com/images/I/51NKoR5aAg...,[],Amazon EG
8,HONOR X9b Dual SIM 12GB RAM+256GB ROM (6.78 In...,HONOR,Android 13.0,12 GB,Snapdragon,12 GB,5.7 Inches,1220 x 2652,None,120 Hz,...,None,None,"EGP14,990.00","EGP23,799.00",EGP,3.8 out of 5 stars,https://www.amazon.eg/-/en/HONOR-Dual-12GB-256...,https://m.media-amazon.com/images/I/817xVeE4Mi...,"[{'reviewer_name': 'Ahmed Elshamy', 'review_ra...",Amazon EG
9,HONOR X8b 8GB RAM 512GB Internal storage- Glam...,HONOR,Android 13.0,None,Snapdragon,512 GB,6.41,None,None,None,...,None,Glamorous Green,"EGP12,130.00",None,EGP,3.0 out of 5 stars,https://www.amazon.eg/-/en/HONOR-512GB-Interna...,https://m.media-amazon.com/images/I/61oExJJ6u8...,[{'reviewer_name': 'هل المشط المتعدد لاكتر من ...,Amazon EG


In [32]:
import pandas as pd
import re
allcolors = [
     "Rose Gold", "Navy Blue","Light Blue","Cobalt Violet","Awesome Graphite","Titanium Black","Wave Green","Emerald Green","Titanium Silver",
    "Midnight Blue", "Forest Green", "Pink", "Purple", "Lavender","Charcoal Ink","Carbon Grey","Glamorous Green","Flowing Silver",
    "Orange", "Bronze", "Copper", "Champagne", "Beige", "Coral", "Midnight Black","Cobalt Violet",
    "Turquoise", "Teal", "Mint Green", "Burgundy", "Aqua", "Cyan", "Lime Green","Rose Red",
    "Space Gray", "Graphite", "Alpine Green", "Starlight", "Pacific Blue", "Ceramic White","Moonlight White"
    "Jet Black", "Matte Black", "Phantom Black", "Phantom Silver", "Mystic Bronze", "Awesome Navy"
    "Mystic Green", "Cosmic Gray", "Cosmic Black", "Pearl White", "Glossy White", "Awesome Lime","Silver Shadow",
    "Aura Glow", "Aurora Blue", "Prism White", "Cloud Pink", "Midnight Green", "Mint","Charcoal",
    "Sunset Gold", "Ocean Blue", "Thunder Purple", "Starry Night", "Twilight", "Clear","Navy","Chartreuse",
    "Gradient Purple", "Gradient Blue", "Gradient Red","Gradient Pink", "Marble White", "Graphite","Violet",
    "Opal White", "Crystal Blue", "Ice Blue", "Sunrise Red", "Fiery Red","Blue","Black", "White", "Gray", "Silver", "Gold", "Red","Yellow", "Green", "Brown"
]
def find_color_in_title(title, allcolors):
    for color in allcolors:
        if color.lower() in title.lower():
            return color
    return None
df_honor['Color'] = df_honor.apply(lambda row: find_color_in_title(row['Title'], allcolors) or row['Color'], axis=1)

In [33]:
df_honor.to_csv('HONOR EG.csv', index = False)

Final Result (Before Cleaning)

In [34]:
df = pd.concat([df_apple, df_samsung, df_oppo, df_xiaomi, df_honor, df_realme], ignore_index=True)

In [35]:
df.to_csv('Final Amazon EG.csv', index = False)